In [2]:
import pandas as pd
import numpy  as np

In [14]:
df = pd.read_csv('./Teste-11-08.csv')
df['AppId'].unique()
df

/home/luan/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (35,43,45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,null,AppName,block_x,compute_version,registers,smem,cmem,number_of_lines_kernel,occupancy,...,start,duration,grid_x,grid_y,grid_z,block_y,block_z,registers_per_thread,static_smem,dynamic_smem
0,0,0,matMul_gpu_sharedmem,8,3.0,32,512,348,15,50,...,0.172444,0.000271811,32,32,1,8,1,22,512,0
1,1,0,matMul_gpu_sharedmem,8,3.0,32,512,348,15,50,...,0.181734,0.002197406,64,64,1,8,1,22,512,0
2,2,0,matMul_gpu_sharedmem,8,3.0,32,512,348,15,50,...,0.187044,0.007433022,96,96,1,8,1,22,512,0
3,3,0,matMul_gpu_sharedmem,8,3.0,32,512,348,15,50,...,0.198440,0.017558,128,128,1,8,1,22,512,0
4,4,0,matMul_gpu_sharedmem,8,3.0,32,512,348,15,50,...,0.200085,0.034728,160,160,1,8,1,22,512,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8454,8454,8,dotProd,1024,5.2,13,4096,348,17,100,...,1.632076,0.028937,245760,1,1,1,1,11,4096,0
8455,8455,8,dotProd,1024,5.2,13,4096,348,17,100,...,1.657057,0.029371,249856,1,1,1,1,11,4096,0
8456,8456,8,dotProd,1024,5.2,13,4096,348,17,100,...,1.774341,0.029843,253952,1,1,1,1,11,4096,0
8457,8457,8,dotProd,1024,5.2,13,4096,348,17,100,...,1.700372,0.030315,258048,1,1,1,1,11,4096,0


In [16]:
df1 = df[['AppName','compute_version','registers','smem','cmem','gpu_id','gpu_name','l1_cache_used','max_clock_rate','num_of_cores','L2','bus','memory_clock','bandwith','GM','num_sm','num_cores_sm','theoretical_flops','AppId','number_of_lines_kernel','occupancy','input_size', 'duration','grid_x','grid_y','block_x','block_y']]
df1 = df1.sort_values(by='AppId', ascending=True)
df1['duration'] = pd.to_numeric(df1['duration'],errors = 'coerce')
df1 = df1.select_dtypes(exclude=['object'])
df1=df1.dropna()

In [17]:
from sklearn import metrics

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
print('n_estimators = 100')
for idApp in range(1,10):
  idApp = float(idApp)
  
  X_train = df1[df1['AppId'] != idApp]
  Y_train = df1[df1['AppId'] != idApp]['duration']
  X_test = df1[df1['AppId'] == idApp]
  Y_test = df1[df1['AppId'] == idApp]['duration']
  
  yTrain = np.log(Y_train + 0.000001)
  yTest = np.log(Y_test + 0.000001)
  xTrain = np.log(X_train + 0.000001)
  xTest = np.log(X_test + 0.000001)
 
  #xTrain, xTest, yTrain, yTest = train_test_split(X, y, train_size=0.8, random_state=42)

  regressor = RandomForestRegressor(n_estimators = 100, random_state = 42)
  regressor.fit(xTrain, yTrain)

  y_pred = regressor.predict(xTest)

  df2=pd.DataFrame({'Actual':yTest, 'Predicted':y_pred})
  print('Application: ', idApp, ' Percentage:',mean_absolute_percentage_error(yTest, y_pred))

  

n_estimators = 100
Application:  1.0  Percentage: 2.7135606652100557
Application:  2.0  Percentage: 0.3404799761927594
Application:  3.0  Percentage: 0.5710001904077597
Application:  4.0  Percentage: 8.316906690243428
Application:  5.0  Percentage: 0.039398224251521986
Application:  6.0  Percentage: 0.048238474870224465
Application:  7.0  Percentage: 0.03115840776367795
Application:  8.0  Percentage: 0.024404798691908924
Application:  9.0  Percentage: 0.15755910249116623


In [58]:
df1['block_x'].unique()

array([ 16, 128])

In [52]:
for i in range(len(y_pred)):
    
    print('Application: ', idApp, ' Percentage:',mean_absolute_percentage_error(df2['Actual'].iloc[i], df2['Predicted'].iloc[i]))
#df2['Actual'].iloc[0]
print(len(y_pred))

Application:  1.0  Percentage: 0.07490718612004961
Application:  1.0  Percentage: 0.47039376571458325
Application:  1.0  Percentage: 0.0164857876852994
Application:  1.0  Percentage: 0.007427393488969376
Application:  1.0  Percentage: 0.02275326019800052
Application:  1.0  Percentage: 0.017806770341580434
Application:  1.0  Percentage: 0.09331631756691165
Application:  1.0  Percentage: 0.0447943189724001
Application:  1.0  Percentage: 0.29899394799115175
Application:  1.0  Percentage: 0.05156048139367624
Application:  1.0  Percentage: 0.37720475766782197
Application:  1.0  Percentage: 0.6915239673172775
Application:  1.0  Percentage: 0.20556080105647223
Application:  1.0  Percentage: 0.19271358793470403
Application:  1.0  Percentage: 4.529637381820226
Application:  1.0  Percentage: 0.15053729980896025
Application:  1.0  Percentage: 0.09154573007905177
Application:  1.0  Percentage: 0.5368465187184346
Application:  1.0  Percentage: 0.2997530027534474
Application:  1.0  Percentage: 0.305